# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [1]:

import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

import _data_utils
import _script_utils

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [4]:
### replace the file path with the new data file
### Latest is from 6/29/23
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_062923_ToDLA.xlsx",))

### last data export 3/9/2023
# proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230309_ToDLA.xlsx"))
                       


In [5]:

# df = utils.read_data_all()
proj = _data_utils.add_new_codes(proj)
    
## function that adds known agency name to df 
df = _script_utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [6]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
1680,2023-06-29,Y233,Surface Transportation Block Grant,5937212,0417000254L,"ALAMITOS ROAD OVER HERBERT CREEK, 1.1 MILES SW OF JUNCTION OF HICK ROAD. (BR 37C0503) BRIDGE REPLACEMENT (TC).",85,Cong Dist 18,Active,"ALAMITOS ROAD OVER HERBERT CREEK, 1.1 MILES SW OF JUNCTION OF HICK ROAD. (BR 37C0503) BRIDGE REPLACEMENT (TC).",16,Right of Way,40000.00,40000.00,L5937MTC,5937.0,Santa Clara County,4.0,Santa Clara County,Metropolitan Transportation Commission,Metropolitan Transportation Commission
376,2022-05-10,Y001,National Highway Performance Program (NHPP),8051641,1117000037S,"IN SAN DIEGO COUNTY IN CHULA VISTA, FROM 0.3 MILE SOUTH OF MAIN STREET UNDERCROSSING TO PALOMAR STREET OVERCROSSING CONVERT POTABLE IRRIGATION SYSTEM",73,Cong Dist 53,Active,"ON STATE ROUTE: 805. IN SAN DIEGO COUNTY IN CHULA VISTA, FROM 0.3 MILE SOUTH OF MAIN STREET UNDERCROSSING TO PALOMAR STREET OVERCROSSING CONVERT POTABLE IRRIGATION SYSTEMS TO RECYCLED WATER.",17,Construction Engineering,623428.24,570793.88,S SANDAG,NaN,California,11.0,San Diego County,NaN,NaN
869,2023-02-20,Y001,National Highway Performance Program (NHPP),5002164,0313000231L,I ST BRIDGE OVER THE SACRAMENTO RIVER BETWEEN SACRAMENTO AND WEST SACRAMENTO. BR. # 22C0153. BRIDGE REPLACEMENT. TOLL CREDITS FOR STBGP FUNDS (TC),67,Cong Dist 6,Active,I ST BRIDGE OVER THE SACRAMENTO RIVER BETWEEN SACRAMENTO AND WEST SACRAMENTO. BR. # 22C0153. BRIDGE REPLACEMENT. TOLL CREDITS FOR STBGP FUNDS (TC),16,Right of Way,6000000.00,5001945.00,L5002SACOG,5002.0,Sacramento,3.0,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments


## Test & Export

In [7]:
### run the data through the rest of the script
### return a dataset that is aggregated at the project and program code

agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/jovyan/data-analyses/dla/iija/_script_utils.py:452: FutureWarning: The default value of regex will change from True to False in a future version.
  df['implementing_agency_locode'] = df['implementing_agency_locode'].str.replace('.0', '')


In [8]:
agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,recipient_project_number,improvement_type,improvement_type_description,program_code,program_code_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
1095,2023-05-15,5129070,Oxnard,L5129SCAG,0712000045L,13|17,Bridge Rehabilitation - Added Capacity|Construction Engineering,Y113,nan,"CHANNEL ISLAND BLVD OVER EDISON CANAL, 0.2 MILES EAST OF HARBOR BLVD, 52C0095 BRIDGE REHABILITATION (NO ADDED LANE CAPACITY)",3257904,|26|,|07|,111,Ventura County,|VEN|,5129,Ventura County Transportation Commission,Southern California Association Of Governments,Bridge Rehabilitation in Oxnard,"Bridge Rehabilitation in Oxnard, part of the nan. (Federal Project ID: 5129070)."
479,2022-09-19,5002200,Sacramento,L5002SACOG,0317000139L,4|17|21,4R - No Added Capacity|Construction Engineering|Safety,Y230|Y240|YS30,Surface Transportation Block Grant|Highway Safety Improvement Program (HSIP),FLORIN ROAD BETWEEN GREENHAVEN DRIVE AND LUTHER DRIVE. CONVERT FROM PEDESTAL MOUNTED TO MAST ARMS AT 7 INTERSECTIONS; INSTALL ADVANCED DETECTION AT F,5602899,|06|,|03|,67,Sacramento County,|SAC|,02,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Road Construction in Sacramento,"Road Construction in Sacramento, part of the Surface Transportation Block Grant|Highway Safety Improvement Program (HSIP). (Federal Project ID: 5002200)."
58,2022-02-22,5060381,Fresno,L5060COFCG,0621000063L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,Y301,Transportation Alternatives,"WB SHIELDS AVENUE FROM BLACKSTONE AVENUE TO FRESNO STREET INSTALL CLASS I TRAIL, PAVED PATH, DROUGHT TOLERANT LANDSCAPING, IRRIGATION, SIGNAGE AND ST",474286,|16|,|06|,19,Fresno County,|FRE|,,Council of Fresno County Governments,Council Of Fresno County Goverments,Install Signage in Fresno,"Install Signage in Fresno, part of the Transportation Alternatives. (Federal Project ID: 5060381)."
874,2023-03-21,6507027,San Bernardino County Transportation Authority,L6507SCAG,0821000039L,14,Bridge Rehabilitation - No Added Capacity,Y110,Bridge Formula Program,"N 1ST AVENUE OVER BNSF RY, UP RR, AMTRAK, 0.2 MI N MAIN ST IN BARSTOW, BR. NO. 54C-0088 REPLACE EXISTING 2 LANE BRIDGE WITH 2 LANE BRIDGE, EXISTING B",5900000,|08|,|08|,71,San Bernardino County,|SBD|,67,NON-RTPA,Southern California Association Of Governments,Replace Bridge in San Bernardino County Transportation Authority,"Replace Bridge in San Bernardino County Transportation Authority, part of the Bridge Formula Program. (Federal Project ID: 6507027)."
1113,2023-05-16,6507003,San Bernardino County Transportation Authority,L6507SCAG,0817000123L,11|15|16|17|43,Bridge Replacement - No Added Capacity|Preliminary Engineering|Right of Way|Construction Engineering|Utilities,Y001,National Highway Performance Program (NHPP),"MOUNT VERNON AVENUE OVERHEAD AT THE BURLINGTON NORTHERN SANTA FE RAILROAD YARD FROM KING STREET TO 5TH STREET, BR. NO. 54C-0066 BRIDGE REPLACEMENT",81076824,|31|,|08|,71,San Bernardino County,|SBD|,67,NON-RTPA,Southern California Association Of Governments,Replace Bridge in San Bernardino County Transportation Authority,"Replace Bridge in San Bernardino County Transportation Authority, part of the National Highway Performance Program (NHPP). (Federal Project ID: 6507003)."


In [9]:
## when grouped by funding program, len is 1612
len(agg)

1465

In [12]:
### pretty print the column names

agg = _script_utils.title_column_names(agg)

In [14]:
#### export new file to the GCS bucket
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_082323_rerun.csv")

### old export
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_agg.csv")


In [13]:
# ###test full. PASS title, no concat and 1241 rows
# full = _script_utils.get_clean_data(df, full_or_agg = 'full')